In [33]:
import pandas as pd
import ast

# Lire le fichier CSV et charger les données dans un DataFrame
df = pd.read_csv('merged_file.csv')

# Supprimer les doublons du DataFrame
df = df.drop_duplicates()

# Supprimer les lignes avec des valeurs manquantes dans la colonne 'Language'
df = df.dropna(subset=['Language'])

# Remplacer les occurrences spécifiques dans la colonne 'contributors'
df['contributors'] = df['contributors'].replace("['The history or contributor list is too large to list contributors for this repository via the API.']", "[]")

# Remplir les valeurs manquantes dans la colonne 'Description' avec 'has no description'
df['Description'] = df['Description'].fillna("has no description")

# Convertir les chaînes de caractères représentant des listes en listes réelles pour les colonnes 'Topics', 'languages' et 'contributors'
df['Topics'] = df['Topics'].apply(ast.literal_eval)
df['languages'] = df['languages'].apply(ast.literal_eval)
df['contributors'] = df['contributors'].apply(ast.literal_eval)


import re

# Définir une fonction pour supprimer les emojis d'une chaîne de caractères
def remove_emojis(text):
    # Expression régulière pour trouver tous les emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # émoticônes
                               u"\U0001F300-\U0001F5FF"  # symboles et pictogrammes
                               u"\U0001F680-\U0001F6FF"  # symboles de transport et cartes
                               u"\U0001F1E0-\U0001F1FF"  # drapeaux (iOS)
                               u"\U00002500-\U00002BEF"  # caractères chinois
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               "]+",
                               flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Appliquer la fonction remove_emojis à la colonne 'Description'
df['Description'] = df['Description'].apply(remove_emojis)

# Le DataFrame 'df' a été modifié avec les transformations précédentes
df


,Name,Description,Stars,URL,Date creation,Dernier modification,Forks,Language,Topics,languages,contributors
0,freeCodeCamp,freeCodeCamp.org's open-source codebase and cu...,369549,https://github.com/freeCodeCamp/freeCodeCamp,2014-12-24T17:49:19Z,2023-07-06T10:34:34Z,32806,TypeScript,"[careers, certification, community, curriculum...","[TypeScript, JavaScript, CSS, MDX, Dockerfile,...","[{'login': 'raisedadead', 'contributions': 142..."
5,public-apis,A collective list of free APIs,247051,https://github.com/public-apis/public-apis,2016-03-20T23:49:42Z,2023-07-06T10:46:14Z,28345,Python,"[api, apis, dataset, development, free, list, ...","[Python, Shell]","[{'login': 'matheusfelipeog', 'contributions':..."
6,developer-roadmap,"Interactive roadmaps, guides and other educati...",243636,https://github.com/kamranahmedse/developer-roa...,2017-03-15T13:45:52Z,2023-07-06T10:42:38Z,34240,TypeScript,"[angular-roadmap, backend-roadmap, blockchain-...","[TypeScript, Astro, JavaScript, CSS]","[{'login': 'kamranahmedse', 'contributions': 1..."
7,system-design-primer,Learn how to design large-scale systems. Prep ...,223929,https://github.com/donnemartin/system-design-p...,2017-02-26T16:15:28Z,2023-07-06T10:35:11Z,39662,Python,"[design, design-patterns, design-system, devel...","[Python, Shell]","[{'login': 'donnemartin', 'contributions': 175..."
8,react,The library for web and native user interfaces,210013,https://github.com/facebook/react,2013-05-24T16:15:54Z,2023-07-06T10:36:32Z,43969,JavaScript,"[declarative, frontend, javascript, library, r...","[JavaScript, HTML, CSS, C++, TypeScript, Coffe...","[{'login': 'zpao', 'contributions': 1778, 'typ..."
...,...,...,...,...,...,...,...,...,...,...,...
2698,editor.js,A block-style editor with clean JSON output,23467,https://github.com/codex-team/editor.js,2015-11-28T20:56:35Z,2023-07-06T10:04:56Z,1791,TypeScript,"[codex-editor, editor, javascript, json, redac...","[TypeScript, HTML, CSS, JavaScript]","[{'login': 'khaydarov', 'contributions': 202, ..."
2699,Hystrix,Hystrix is a latency and fault tolerance libra...,23465,https://github.com/Netflix/Hystrix,2012-11-19T20:14:46Z,2023-07-06T06:42:21Z,4710,Java,[],"[Java, Clojure, CSS, Groovy]","[{'login': 'mattrjacobs', 'contributions': 538..."
2700,awesome-swift,A collaborative list of awesome Swift librarie...,23463,https://github.com/matteocrippa/awesome-swift,2014-07-10T14:04:09Z,2023-07-06T06:29:10Z,3051,Ruby,"[awesome, awesome-list, awesome-lists, awesome...",[Ruby],"[{'login': 'READMEbot', 'contributions': 733, ..."
2701,pkg,Package your Node.js project into an executable,23454,https://github.com/vercel/pkg,2016-08-08T19:41:59Z,2023-07-06T10:29:54Z,1005,JavaScript,"[binaries, cli, compiler, nodejs]","[JavaScript, TypeScript, CSS, HTML, Pug, Coffe...","[{'login': 'igorklopov', 'contributions': 887,..."


In [34]:
df.columns

Index(['Name', 'Description', 'Stars', 'URL', 'Date creation',
       'Dernier modification', 'Forks', 'Language', 'Topics', 'languages',
       'contributors'],
      dtype='object')

In [35]:
import mysql.connector

# Établir une connexion à la base de données MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="github"
)

# Créer un curseur
cursor = conn.cursor()

# Code SQL pour créer les tables
sql_code = """
DROP TABLE IF EXISTS RepoTopic;
DROP TABLE IF EXISTS RepoContributor;
DROP TABLE IF EXISTS RepoLanguage;
DROP TABLE IF EXISTS Language;
DROP TABLE IF EXISTS Topic;
DROP TABLE IF EXISTS Contributor;
DROP TABLE IF EXISTS Repo;

CREATE TABLE Repo (
    id INT PRIMARY KEY AUTO_INCREMENT,
    Name VARCHAR(255),
    Description LONGTEXT,
    Stars INT,
    URL VARCHAR(255),
    DateCreation DATE,
    LastModification DATE,
    Forks INT
);

CREATE TABLE Contributor (
    id INT PRIMARY KEY AUTO_INCREMENT,
    login VARCHAR(255)
);

CREATE TABLE Topic (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255)
);

CREATE TABLE Language (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255)
);

CREATE TABLE RepoLanguage (
    repo_id INT,
    language_id INT,
    FOREIGN KEY (repo_id) REFERENCES Repo(id),
    FOREIGN KEY (language_id) REFERENCES Language(id),
    is_main BOOLEAN
);

CREATE TABLE RepoContributor (
    repo_id INT,
    contributor_id INT,
    contributions INT,
    FOREIGN KEY (repo_id) REFERENCES Repo(id),
    FOREIGN KEY (contributor_id) REFERENCES Contributor(id)
);

CREATE TABLE RepoTopic (
    topic_id INT,
    repo_id INT,
    FOREIGN KEY (topic_id) REFERENCES Topic(id),
    FOREIGN KEY (repo_id) REFERENCES Repo(id)
);
"""

# Exécuter le code SQL
for statement in sql_code.split(";"):
    if statement.strip() != "":
        cursor.execute(statement)

# Valider les changements dans la base de données
conn.commit()

# Fermer le curseur et la connexion
cursor.close()
conn.close()

print('Terminé')

Terminé


In [36]:
import mysql.connector
from datetime import datetime

# Établir une connexion à la base de données MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="github"
)

# Créer un curseur
cursor = conn.cursor()

# Insérer des données à partir du DataFrame
for _, row in df.iterrows():
    # Insérer les données Repo
    repo_data = {
        "Name": row['Name'],
        "Description": row['Description'],
        "Stars": row['Stars'],
        "URL": row['URL'],
        "DateCreation": datetime.strptime(row['Date creation'], '%Y-%m-%dT%H:%M:%SZ').date(),
        "LastModification": datetime.strptime(row['Dernier modification'], '%Y-%m-%dT%H:%M:%SZ').date(),
        "Forks": row['Forks']
    }
    
    print(repo_data['Description'])

    repo_insert_query = """
    INSERT INTO Repo (Name, Description, Stars, URL, DateCreation, LastModification, Forks)
    VALUES (%(Name)s, %(Description)s, %(Stars)s, %(URL)s, %(DateCreation)s, %(LastModification)s, %(Forks)s)
    """
    cursor.execute(repo_insert_query, repo_data)

    repo_id = cursor.lastrowid
    
    # *---- Début Insérer des Langages ----*
    languages = row['languages']
    language_ids = {}

    language_insert_query = """
    INSERT INTO Language (name)
    VALUES (%s)
    """
    for language in languages:
        language_id_query = "SELECT id FROM Language WHERE name = %s"
        cursor.execute(language_id_query, (language,))
        result = cursor.fetchone()
        if result:
            language_id = result[0]
        else:
            is_main = (language == row['Language'])
            cursor.execute(language_insert_query, (language,))
            language_id = cursor.lastrowid
        language_ids[language] = language_id
        
            # Insérer RepoLanguage
    repo_language_insert_query = """
    INSERT INTO RepoLanguage (repo_id, language_id)
    VALUES (%s, %s)
    """

    for language in language_ids:
        cursor.execute(repo_language_insert_query, (repo_id, language_ids[language]))
    
    # !---- Fin Insérer des Langages ----!
    
    # *---- Début Insérer des Sujets ----*
    topics = row['Topics']
    topic_ids = {}

    topic_insert_query = """
    INSERT INTO Topic (name)
    VALUES (%s)
    """
    for topic in topics:
        topic_id_query = "SELECT id FROM Topic WHERE name = %s"
        cursor.execute(topic_id_query, (topic,))
        result = cursor.fetchone()
        if result:
            topic_id = result[0]
        else:
            cursor.execute(topic_insert_query, (topic,))
            topic_id = cursor.lastrowid
        topic_ids[topic] = topic_id

            # Insérer RepoTopic
    repo_topic_insert_query = """
    INSERT INTO RepoTopic (repo_id, topic_id)
    VALUES (%s, %s)
    """

    for topic in topic_ids:
        cursor.execute(repo_topic_insert_query, (repo_id, topic_ids[topic]))

    # !---- Fin Insérer des Sujets ----!

    # *---- Début Insérer des Contributeurs ----*
    contributors = row['contributors']
    contributor_ids = {}

    contributor_insert_query = """
    INSERT INTO Contributor (login)
    VALUES (%s)
    """

    for contributor in contributors:
        login = contributor['login']
        contributions = contributor['contributions']
        contributor_id_query = "SELECT id FROM Contributor WHERE login = %s"
        cursor.execute(contributor_id_query, (login,))
        result = cursor.fetchone()
        if result:
            contributor_id = result[0]
        else:
            cursor.execute(contributor_insert_query, (login,))
            contributor_id = cursor.lastrowid
        contributor_ids[login] = {"contributor_id":contributor_id,"contributions":contributions}



    # Insérer RepoContributor
    repo_contributor_insert_query = """
    INSERT INTO RepoContributor (repo_id, contributor_id, contributions)
    VALUES (%s, %s, %s)
    """
    for contributor in contributor_ids:
        cursor.execute(repo_contributor_insert_query, (repo_id, contributor_ids[contributor]['contributor_id'], contributor_ids[contributor]['contributions']))

    # ---- Fin Insérer des Contributeurs ---!

    print("_", _)
    if _ == 20:
        break

# Valider les changements dans la base de données
conn.commit()

# Fermer le curseur et la connexion
cursor.close()
conn.close()

freeCodeCamp.org's open-source codebase and curriculum. Learn to code for free.
_ 0
A collective list of free APIs
_ 5
Interactive roadmaps, guides and other educational content to help developers grow in their careers.
_ 6
Learn how to design large-scale systems. Prep for the system design interview.  Includes Anki flashcards.
_ 7
The library for web and native user interfaces
_ 8
This is the repo for Vue 2. For Vue 3, go to https://github.com/vuejs/core
_ 10
An Open Source Machine Learning Framework for Everyone
_ 11
A curated list of awesome Python frameworks, libraries, software and resources
_ 12
 Algorithms and data structures implemented in JavaScript with explanations and links to further readings
_ 13
The most popular HTML, CSS, and JavaScript framework for developing responsive, mobile first projects on the web.
_ 16
All Algorithms implemented in Python
_ 17
   A delightful community-driven (with 2,100+ contributors) framework for managing your zsh configuration. Includes 300

In [38]:
import mysql.connector

# Établir une connexion à la base de données MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="github"
)

# Créer un curseur pour exécuter des requêtes
cursor = conn.cursor()

# Requête SQL à exécuter
query = """
SELECT c.login AS Contributor, COUNT(rc.contributions) AS Contributions, GROUP_CONCAT(r.Name) AS ContributedRepos FROM Contributor c JOIN RepoContributor rc ON c.id = rc.contributor_id JOIN Repo r ON rc.repo_id = r.id GROUP BY c.login ORDER BY COUNT(rc.contributions) DESC LIMIT 10;
"""

# Exécuter la requête SQL
cursor.execute(query)

# Récupérer les résultats de la requête
results = cursor.fetchall()

# Afficher les résultats
for row in results:
    contributor = row[0]
    contributions = row[1]
    contributed_repos = row[2]
    print(f"Contributor: {contributor}, Contributions: {contributions}, Contributed Repos: {contributed_repos}")

# Fermer le curseur et la connexion
cursor.close()
conn.close()


Contributor: dependabot[bot], Contributions: 4, Contributed Repos: freeCodeCamp,react,vue,bootstrap
Contributor: cclauss, Contributions: 2, Contributed Repos: system-design-primer,Python
Contributor: a14n, Contributions: 1, Contributed Repos: flutter
Contributor: aam, Contributions: 1, Contributed Repos: flutter
Contributor: abarth, Contributions: 1, Contributed Repos: flutter
Contributor: abhisekp, Contributions: 1, Contributed Repos: freeCodeCamp
Contributor: acdlite, Contributions: 1, Contributed Repos: react
Contributor: ad-m, Contributions: 1, Contributed Repos: system-design-primer
Contributor: zpao, Contributions: 1, Contributed Repos: react
Contributor: adnn-alc, Contributions: 1, Contributed Repos: developer-roadmap


In [39]:
import mysql.connector

# Établir une connexion à la base de données MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="github"
)

# Créer un curseur pour exécuter des requêtes
cursor = conn.cursor()

# Requête SQL à exécuter
query = """
SELECT Language.name, COUNT(*) AS LanguageCount
FROM RepoLanguage
INNER JOIN Language ON RepoLanguage.language_id = Language.id
GROUP BY Language.name
ORDER BY LanguageCount DESC
LIMIT 10;
"""

# Exécuter la requête SQL
cursor.execute(query)

# Récupérer les résultats de la requête
results = cursor.fetchall()

# Afficher les résultats
for row in results:
    language_name = row[0]
    language_count = row[1]
    print(f"Language: {language_name}, Count: {language_count}")

# Fermer le curseur et la connexion
cursor.close()
conn.close()


Language: Shell, Count: 10
Language: Python, Count: 8
Language: JavaScript, Count: 7
Language: CSS, Count: 6
Language: HTML, Count: 6
Language: Makefile, Count: 5
Language: TypeScript, Count: 4
Language: C, Count: 4
Language: C++, Count: 4
Language: Dockerfile, Count: 3
